<a href="https://colab.research.google.com/github/HongMinhNhat35/Industrial_Training/blob/main/ELEC_E8130_Nonlinear_Control_Design_and_Analysis_D_HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sympy as sp
import numpy as np

!pip install control
import control as cl
import plotly.graph_objects as go

# Clear installation output to avoid clutter.
from IPython.display import clear_output

clear_output()

In [2]:
# USEFUL FUNCTIONS

# visualization of the value distribution of a real-valued function f(x,y)

f = lambda x,y: x**2 + y**2

X, Y = np.meshgrid(np.linspace(-5, 5, 100), np.linspace(-5, 5, 100))
Z = f(X, Y).reshape(X.shape)

fig = go.Figure(data=[
    go.Surface(z=Z, x=X, y=Y, showscale=True, opacity=0.5,
               colorbar=dict(lenmode='fraction', len=0.6, thickness=18)),
])

fig.update_layout(title='Quadratic function', autosize=True, width=800, height=600, margin=dict(l=0, r=0, b=15, t=30))
fig.update_layout(scene=dict(
    xaxis_title='theta',
    yaxis_title='d_theta',
    zaxis_title='V(q)'))
fig.update_layout(scene_aspectmode='cube')
fig.update_coloraxes(colorbar_xpad=0)
fig.show()

# Cart-Pole system

Consider the following dynamic of the cart-pole system

$
\dot{q} = f(q,u) =
\begin{bmatrix}
    \dot{x} \\
    \frac{u + m \sin{\theta}(l\dot{\theta}^2-g\cos{\theta})}{M+m \sin^2{\theta}} \\
    \dot{\theta} \\
    \frac{u\cos{\theta} + ml\dot{\theta}^2\cos{\theta}\sin{\theta} - (M+m)g \sin{\theta}}{l(M+m\sin^2{\theta})}
\end{bmatrix}
$

with $M=3, m=2, l=1, g=9.8065$

<div>
<img src="https://drive.google.com/uc?export=view&id=1TFoYP3r5yUZebUBCfQuZVJWkBqhZ9txB" width="400"/>
</div>

In [33]:
x, dx, theta, d_theta = sp.symbols('x dx theta d_theta')
u = sp.symbols('u:1')
q = [x, dx, theta, d_theta]

mc = 3
mp = 2
l = 1
g = 9.8065


def dq():
    frac0 = u[0] + mp * sp.sin(q[2]) * (l * q[3] ** 2 - g * sp.cos(q[2]))
    frac1 = mc + mp * sp.sin(q[2]) ** 2

    dd_x = frac0 / frac1

    frac0 = (u[0] * sp.cos(q[2]) + mp * l * q[3] ** 2 * sp.cos(q[2]) * sp.sin(q[2]) - (mc + mp) * g * sp.sin(q[2]))
    frac1 = l * (mc + mp * sp.sin(q[2]) ** 2)

    dd_theta = frac0 / frac1

    return sp.Matrix([q[1], dd_x, q[3], dd_theta])


dyn = dq()  # this is the dynamic of the system

## Question 1

Consider zero control input i.e., $u=0$. Notice that $q_e=[0,0,0,0]^T$ is an equilibrium point of the uncontrolled system $\dot{q} = f(q,0)$, corresponding to the upright position of the pole. Is $q_e$ stable or unstable?

 ***HINT:*** Compute $A = \frac{\partial}{\partial q} f(q,0)$ at $q = q_e$ and check if $A$ is a Hurwitz matrix. Directly apply Theorem 3.7 from Khalil textbook.



In [12]:
uq = np.zeros(len(u))
for i in range(len(u)):
    dyn = dyn.subs(u[i], uq[i])

# ---------------------------------------------------------------
# YOUR CODE HERE
# A = ?
# Check eigenvalues of A
# Conclude about stability/unstability
A = dyn.jacobian(q)

equil_pt = {x: 0, dx: 0, theta: 0, d_theta: 0}
A_at_eq = A.subs(equil_pt)
A_num = np.array(A_at_eq).astype(np.float64)
eigenvalues = np.linalg.eigvals(A_num)

print("Eigenvalues:", eigenvalues)
print(A)
if np.all(np.real(eigenvalues) < 0):
    print("The system is stable.")
else:
    print("The system is unstable.")



# ---------------------------------------------------------------

Eigenvalues: [0.+0.j         0.+0.j         0.+4.04279194j 0.-4.04279194j]
Matrix([[0, 1, 0, 0], [0, 0, 2*(d_theta**2 - 9.8065*cos(theta))*cos(theta)/(2*sin(theta)**2 + 3) - 8*(d_theta**2 - 9.8065*cos(theta))*sin(theta)**2*cos(theta)/(2*sin(theta)**2 + 3)**2 + 19.613*sin(theta)**2/(2*sin(theta)**2 + 3), 4*d_theta*sin(theta)/(2*sin(theta)**2 + 3)], [0, 0, 0, 1], [0, 0, -4*(2*d_theta**2*sin(theta)*cos(theta) - 49.0325*sin(theta))*sin(theta)*cos(theta)/(2*sin(theta)**2 + 3)**2 + (-2*d_theta**2*sin(theta)**2 + 2*d_theta**2*cos(theta)**2 - 49.0325*cos(theta))/(2*sin(theta)**2 + 3), 4*d_theta*sin(theta)*cos(theta)/(2*sin(theta)**2 + 3)]])
The system is unstable.


## Question 2

Now we consider a linear feedback controller $u(q)=-K(q-q_e)$. The point $q_e=[0,0,0,0]^T$ is still an equilibrium point for the closed-loop system $\dot{q} = f(q,u(q))$. For $K=[1,3.371, 1.781, 1.891]$, determine if the equilibrium $q_e$ is now stable or unstable.

 ***HINT:*** Use the same steps as before. Start by computing $A = \frac{\partial}{\partial q} f(q,u(q))$ at $q = q_e$.

In [11]:
K = np.array([[1, 3.371, 1.781, 1.891]], dtype=float)
qe = np.array([0, 0, 0, 0], dtype=float)

uq = -sp.Matrix(K) @ sp.Matrix(q - qe)
for i in range(len(u)):
    dyn = dyn.subs(u[i], uq[i])

# ---------------------------------------------------------------
# YOUR CODE HERE
# A = ?
# Check eigenvalues of A
# Conclude about stability/unstability
A = dyn.jacobian(q)

equil_pt = {x: 0, dx: 0, theta: 0, d_theta: 0}
A_at_eq = A.subs(equil_pt)
A_num = np.array(A_at_eq).astype(np.float64)
eigenvalues = np.linalg.eigvals(A_num)

print("Eigenvalues:", eigenvalues)

if np.all(np.real(eigenvalues) < 0):
    print("The system is stable at equilibrium point.")
else:
    print("The system is unstable at equilibrium point.")


# ---------------------------------------------------------------

Eigenvalues of the Jacobian matrix at equilibrium: [0.+0.j         0.+0.j         0.+4.04279194j 0.-4.04279194j]
The system is unstable at the equilibrium point.


## Question 3

Using the $A$ matrix from above, compute a quadratic Lyapunov function of the form $V(q) = q^TPq$ by solving the Lyapunov equation $A^TP+PA=-Q$ with symmetric matrix $Q \succ 0$. You can choose any $Q \succ 0$ you like.

***HINT:*** check [this documentation ](https://python-control.readthedocs.io/en/latest/index.html)to find appropriate API

or can use for example, scipy.linalg

In [31]:
# ---------------------------------------------------------------
# YOUR CODE HERE
# Pick any positive deninite matrix Q
# Solve for P
# Create function V(q)
Q=np.array([[4,1,1,1],
            [1,3,0.5,0.5],
            [1,0.5,2,0.5],
            [1,0.5,0.5,3]])
P=cl.lyap(A_num,Q)
def V(q):
    q = np.array(q)
    return np.dot(q.T, np.dot(P, q))
print(V(qe))
# ---------------------------------------------------------------

0.0


## Optional bonus question

Optional bonus question [Extra 10 points]: Set the initial condition for $x$ and $\dot{x}$ to be zero. Through simulations, estimate the region of attraction of $q_e$ in the $\theta-\dot{\theta}$ plane. Plot $\dot{V}(q)$ in the $\theta-\dot{\theta}$ by fixing $x=0,\, \dot{x} = 0.$ Notice that $\dot{V}(q)$ has negative values in some small neighborhood of $q_e.$

**HINT:** For different values of $q_3,q_4$, start from $(0,0,q_3,q_4)$ and check if system converges to $q_e.$

In [32]:
# ---------------------------------------------------------------
# YOUR CODE HERE (The functions related to visualization have already been provided at the beginning.)

f = lambda a, b: np.array([V([0, 0, a_val, b_val]) for a_val, b_val in zip(a.ravel(), b.ravel())])

X, Y = np.meshgrid(np.linspace(-50, 50, 100), np.linspace(-50, 50, 100))
Z = f(X, Y).reshape(X.shape)

fig = go.Figure(data=[
    go.Surface(z=Z, x=X, y=Y, showscale=True, opacity=0.5,
               colorbar=dict(lenmode='fraction', len=0.6, thickness=18)),
])

fig.update_layout(title='Quadratic function', autosize=True, width=800, height=600, margin=dict(l=0, r=0, b=15, t=30))
fig.update_layout(scene=dict(
    xaxis_title='theta',
    yaxis_title='d_theta',
    zaxis_title='V(q)'))
fig.update_layout(scene_aspectmode='cube')
fig.update_coloraxes(colorbar_xpad=0)
fig.show()



# ---------------------------------------------------------------

## Animation of this Cart-Pole system

In [ ]:
# animation
import matplotlib.pyplot as plt
from matplotlib.animation import FFMpegWriter
from matplotlib.patches import Rectangle
from scipy.integrate import odeint

# Setup Figure:
fig, ax = plt.subplots()
p, = ax.plot([], [], color='royalblue')
min_lim = -5
max_lim = 5
ax.axis('equal')
ax.set_xlim([min_lim, max_lim])
ax.set_ylim([min_lim, max_lim])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Cart-Pole Simulation:')
title = "simulation"


# ---------------------------------------------------------------
# Test the performance of this LQR controller on the system with various initial states.
# Two options provide below

T = 15 # annimaiton time
x_init = np.array([-1, -0.1, np.pi / 8, -0.2])
#x_init = np.array([-1, -0.1, np.pi * 7 / 8, -0.2])

# ---------------------------------------------------------------


def simulate(ff, pts, T, dt, round_up=True):
    if not isinstance(pts, np.ndarray):
        pts = np.asarray(pts, dtype=float)
    if pts.ndim == 1:
        pts = pts.reshape(1, -1)
    assert pts.ndim == 2  # N*n matrix, N n-dimensional points
    assert T > 0 and T >= dt
    T = np.ceil(T / dt) * dt if round_up else np.floor(T / dt) * dt
    steps = np.ceil(T / dt).astype(int)

    def ode_fx(x, t):
        x = x.reshape(pts.shape)
        y = ff(*x.T).squeeze(axis=1).T
        return y.flatten()

    traj_data = []

    x_cur = pts

    for i in range(steps):
        x_next = odeint(ode_fx, x_cur.flatten(), np.linspace(0, dt, 5))[-1].reshape(pts.shape)
        traj_data.append(x_next)
        x_cur = x_next

    traj_data = np.stack(traj_data)

    return traj_data

ff = sp.lambdify(q,dyn)


traj = simulate(ff, x_init.reshape(1,-1), T, 0.01)
tt = np.linspace(0, T, traj.shape[0])

# Setup Animation Writer:
FPS = 20
dt = 0.01  # exactly same the simulation step
sample_rate = int(1 / (dt * FPS))  # Real Time Playback
dpi = 300
writerObj = FFMpegWriter(fps=FPS)

# Initialize Patch: (Cart)
width = 0.4  # Width of Cart
height = width / 2  # Height of Cart
xy_cart = (x_init[0] - width / 2, x_init[1] - height / 2)  # Bottom Left Corner of Cart
r = Rectangle(xy_cart, width, height, color='cornflowerblue')  # Rectangle Patch
ax.add_patch(r)  # Add Patch to Plot

# Draw the Ground:
ground = ax.hlines(-height / 2, min_lim, max_lim, colors='black')
height_hatch = 0.25
width_hatch = max_lim - min_lim
xy_hatch = (min_lim, 0 - height / 2 - height_hatch)
ground_hatch = Rectangle(xy_hatch, width_hatch, height_hatch, facecolor='None', linestyle='None', hatch='/')
ax.add_patch(ground_hatch)

# Animate:
with writerObj.saving(fig, title + ".mp4", dpi):
    for i in range(0, len(tt), sample_rate):
        this_data = traj[i, -1, :]
        # Update Pendulum Arm:
        x_pole = this_data[0] + l * np.sin(this_data[2])
        y_pole = 0 + l * np.cos(this_data[2])
        x_pole_arm = [this_data[0], x_pole]
        y_pole_arm = [0, y_pole]
        p.set_data(x_pole_arm, y_pole_arm)
        # Update Cart Patch:
        r.set(xy=(this_data[0] - width / 2, 0 - height / 2))
        # Update Drawing:
        fig.canvas.draw();
        # Save Frame:
        writerObj.grab_frame();
plt.close()
from IPython.display import Video

Video('./simulation.mp4', embed=True, width=640, height=480)